In [1]:
import glob #---------------------------------------------------- To read the files or folders in a system directory
import re #------------------------------------------------------ To replace characters in a string
import numpy    as np
import datetime
import warnings #------------------------------------------------ To suppress warnings
import h5py #---------------------------------------------------- To read hdf5 files
import pandas   as pd
import pyresample
from   scipy                         import spatial #-------------------------------------- To extract the values and indices of k nearest neighbors
from   ast                           import literal_eval #----------------------------------- For literal evaluation of a string to extract python objects
from   pyproj                        import Proj, transform #----------------------------- To interconvert different projections
from   netCDF4                       import Dataset #------------------------------------ To read nc , nc4 and hdf4 files
from   photutils.utils               import ShepardIDWInterpolator as idw #------ To use Shepard's Inverse Distance Weighing Interpolation tool
from   IPython.core.interactiveshell import InteractiveShell  #---- Output all jupyter lab inputs instead of the last one
from   IPython.display               import Markdown, display

InteractiveShell.ast_node_interactivity = "all"
warnings.simplefilter('ignore','always')
def printmd(string):
    display(Markdown(string))

/home/kaushal/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/kaushal/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
### Define Non-Iterative Functions and Variables

in_proj  = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext') #------ Specify input projection
out_proj = Proj(init='epsg:4326') #----------------------------------------- Specify output projection


def Times(x): #------------------------------------------Extract Time from sounding ID. NOTE the time format is HH:MM:SSSS
    y   = str(x)
    yy  = y[8:]
    yyy = '{}:{}:{}'.format(yy[:2], yy[2:4], yy[4:])
    return yyy

hours = [0, 3000000, 6000000, 9000000, 12000000, 15000000, 18000000, 21000000, 23595900]
def f(x): #---------------------------------------------Extract the hour interval of sif time (which  is in seconds)
    for i in range(len(hours)):
        if (x>hours[i]) and (x<hours[i+1]):
            lb = hours[i]
            ub = hours[i+1]
            return lb,ub
            break

def format_time(t): #----------------------------------- Format the time into HH:MM:SSSS for the raw format HH:MM:SSSSSSSS
    s = t
    return s[:-4]

def nn(latitude_list,longitude_list,target): #---------- Find the index of nearest neighbor (NOTE: absolute difference)
    target_lat, target_lon = target[1], target[0]
    d = [abs(latitude-target_lat) + abs(longitude-target_lon) for latitude,longitude in zip(latitude_list,longitude_list)]
    return np.argmin(d)

data = np.genfromtxt('sn_bound_10deg.txt', skip_header = 7, skip_footer = 3)
def tile_finder(Lat,Lon): #----------------------------- Find modis tile numbers in which the argument lat,lon lies
    in_tile = False
    i = 0
    while(not in_tile):
        in_tile = Lat >= data[i, 4] and Lat <= data[i, 5] and Lon >= data[i, 2] and Lon <= data[i, 3]
        i += 1
    V = str(int(data[i-1, 0])).zfill(2)
    H = str(int(data[i-1, 1])).zfill(2)
    return H,V

def extract_pixel_coordinates(ULx,Uly,LRx,LRy,shape):
    x        = np.linspace(ULx, LRx, shape[0], endpoint=False) + abs((ULx-LRx)/(2*shape[0]))
    y        = np.linspace(ULy, LRy, shape[0], endpoint=False) - abs((ULy-LRy)/(2*shape[0]))
    xx, yy   = np.meshgrid(x,y)
    xs       = xx.flatten()
    ys       = yy.flatten()
    plon, plat = transform(in_proj, out_proj, xs, ys)
    return plon, plat

def temporal_interpolation(time1,val1,time2,val2,timeX):
    df    = pd.DataFrame( [(time1, val1) , (time2, val2)] , columns=['Times','Values'] ) 
    df    = df.set_index('Times')
    df    = pd.Series(df['Values'], index=df.index)
    df.index = pd.to_datetime(df.index)
    inter = df.resample('S').interpolate(method='linear')
    valX  = inter.loc[timeX]
    return valX

sif_file_list     = glob.glob('OCO2_sif/*.nc4')    #--------------------------------------------- List of all OCO2 files
calipso_file_list = glob.glob('OCO2_calipso/*.h5') #--------------------------------------------- List of all OCO2-CALIPSO files
fpar_file_list    = glob.glob('MCD15A3H/*.hdf')    
par_folder_list   = glob.glob('MCD18A2/*')
ref_folder_list   = glob.glob('MCD43A4/*')

data              = np.genfromtxt('sn_bound_10deg.txt', skip_header = 7, skip_footer = 3) #------ File having tile numbers and IDs

In [26]:
par       = Dataset('MCD18A2/121/MCD18A2.A2018121.h11v04.006.2019091181205.hdf', mode='r')
#par_date  = datetime.datetime.strptime(par_file.split('.')[1][1:], '%Y%j').strftime("%Y-%m-%d")
gmt_0000  = par.variables['GMT_0000_PAR'][:].flatten() #-----------------------------------------Read par 3-hourly variables
gmt_0300  = par.variables['GMT_0300_PAR'][:].flatten()
gmt_0600  = par.variables['GMT_0600_PAR'][:].flatten()
gmt_0900  = par.variables['GMT_0900_PAR'][:].flatten()
gmt_1200  = par.variables['GMT_1200_PAR'][:].flatten()
gmt_1500  = par.variables['GMT_1500_PAR'][:].flatten()
gmt_1800  = par.variables['GMT_1800_PAR'][:].flatten()
gmt_2100  = par.variables['GMT_2100_PAR'][:].flatten()

shape=(240,240)
struct    = getattr(par, 'StructMetadata.0')
struct1   = struct[struct.find('UpperLeftPointMtrs'): struct.find('LowerRightMtrs')][19:-3] #-------------]
struct2   = struct[struct.find('LowerRightMtrs')    : struct.find('Projection')    ][15:-3] #-------------] Extract upper right and lower left
ULx, ULy  = literal_eval(struct1) #-----------------------------------------------------------------------] coordinates of the tile of opened par file
LRx, LRy  = literal_eval(struct2) #-----------------------------------------------------------------------]
par_lon,par_lat = extract_pixel_coordinates(ULx,ULy,LRx,LRy,shape) #----------- Extract the par lat,lon meshgrid in proper projection 
#tree      = spatial.KDTree(   list(  zip(par_lon, par_lat) )) #---------------- Create a nearest neighbor spatial tree

In [28]:
par_lon = par_lon.reshape(240,240)
par_lat = par_lat.reshape(240,240)

In [29]:
gmt_1500 = gmt_1500.reshape(240,240)

In [7]:
#lon_curv  = par_lon[5000:10000]
#lat_curv  = par_lat[5000:10000]

#gmt_1500  = par.variables['GMT_1500_PAR'][:].flatten()
#lon       = np.linspace(-180,180,240,endpoint=True)#par_lon
#lat       = np.linspace(-90,90,240,endpoint=True)#par_lat
#lon

# get 2D versions of the lat and lon variables (note the -180 here!)
#lon2d, lat2d = np.meshgrid(lon, lat)

In [30]:
tar_lon = par_lon[100].reshape(12,20)
tar_lat = par_lat[100].reshape(12,20)

In [31]:
source_def = pyresample.geometry.SwathDefinition(lons=par_lon, lats=par_lat)
target_def = pyresample.geometry.SwathDefinition(lons=tar_lon, lats=tar_lat)

In [32]:
result = pyresample.bilinear.resample_bilinear(gmt_1500, source_def, target_def, radius=50e3, neighbours=32, nprocs=1, fill_value=None, reduce_data=True, segments=None, epsilon=0)

AttributeError: 'SwathDefinition' object has no attribute 'proj_str'

In [16]:
import numpy as np
from pyresample import image, geometry
area_def = geometry.AreaDefinition('areaD', 'Europe (3km, HRV, VTC)', 'areaD',
                               {'a': '6378144.0', 'b': '6356759.0',
                                'lat_0': '50.00', 'lat_ts': '50.00',
                                'lon_0': '8.00', 'proj': 'stere'},
                               800, 800,
                               [-1370912.72, -909968.64,
                                1029087.28, 1490031.36])
data = np.fromfunction(lambda y, x: y*x, (50, 10))
lons = np.fromfunction(lambda y, x: 3 + x, (50, 10))
lats = np.fromfunction(lambda y, x: 75 - y, (50, 10))
swath_def = geometry.SwathDefinition(lons=lons, lats=lats)
swath_con = image.ImageContainerNearest(data, swath_def, radius_of_influence=5000)
area_con = swath_con.resample(area_def)
result = area_con.image_data

In [14]:
result

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [111]:
indices = np.arange(0,57599,1)
nns = []
#par_lon[indexx], par_lat[indexx]
#gmt_1500[indexx]
for indexx in indices:
    D = []
    n = []
    for iii in np.arange(2,50,1):
        target = (par_lon[indexx], par_lat[indexx])
        neighbors          = tree.query([(par_lon[indexx], par_lat[indexx])], k=iii)[1][0] #---Find k=3 nearest spatial neighbors indices at target
        lon_for_idw        = [par_lon[i] for i in neighbors] #----------------------------Extract longitudes at these indices
        lat_for_idw        = [par_lat[i] for i in neighbors] #----------------------------Extract latitudes also
        coors_for_idw      = [(i,j) for i,j in zip(lat_for_idw,lon_for_idw)] #---------Create a list of coordinates (lat lon tuples)

        gmt_1500n       = [gmt_1500[i] for i in neighbors]
        func_gmt_1500   = idw(coors_for_idw, gmt_1500n)
        interp_gmt_1500 = func_gmt_1500(target)
        diff            = abs(gmt_1500[indexx] - interp_gmt_1500)*100/gmt_1500[indexx]
        D.append(diff)
        n.append(iii)
  
    arg = np.argmin(np.array(D))
    nns.append(np.array(n)[arg])
nns = np.array(nns)

In [113]:
a = nns
print("Size of original array:",len(a))
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Size of original array: 57599
Frequency of unique values of the said array:
[[   2    3    4    5    6    7    8    9   10   11   12   13   14   15
    16   17   18   19   20   21   22   23   24   25   26   27   28   29
    30   31   32   33   34   35   36   37   38   39   40   41   42   43
    44   45   46   47   48   49]
 [9661 8402 6135 4446 3598 3578 3026  750 1692  416 1874 1192  659  760
   539  628  515  555  564  431  455  435  472  287  457   32  460   15
   502   26  392  141  394  265  325  321  226  360  344  148  341   94
   327  139  471  140  512   97]]


In [ ]:
for sub in range(len(sif_time)): #--------------------------Loop through the list of a dataframe cell (grouped dataframe cells consist of lists of values)
    target          = (sif_lat[sub] , sif_lon[sub])  #---------------------------------------------------Make sif coordinates target for spatial interpolation
    sif_time_sub    = re.sub(':', '', sif_time[sub]) #---------------------------------------------------Remove : from sif_time
    timeX           = pd.to_datetime(sif_time_sub.ljust(8, "0"), format="%H%M%S%f").strftime("%H:%M:%S")#--Change resolution from SSSS to SS for faster interpolation
    lower_bound_key = str('interp_gmt_')+str(f(int(sif_time_sub))[0]).zfill(8)[:4] #---Find lower bound of interval in which sif time lies

    if f(int(sif_time_sub))[1] == 23595900: #-----------------------------------------------If upper bound is greater than 2100 hours that is 235959
        upper_bound_key = str('interp_gmt_')+str(f(int(sif_time_sub))[1]).zfill(8)[:6] #----upper key becomes interp_gmt_235959
    else:                                                                              #----else                   
        upper_bound_key = str('interp_gmt_')+str(f(int(sif_time_sub))[1]).zfill(8)[:4] #----upper key is as it is (interp_gmt_1200,1500,1800,etc)

    neigh5          = tree.query([(sif_lon[sub], sif_lat[sub])], k=10)[1][0] #---Find k=3 nearest spatial neighbors indices at target
    lon_for_idw     = [par_lon[i] for i in neigh5] #----------------------------Extract longitudes at these indices
    lat_for_idw     = [par_lat[i] for i in neigh5] #----------------------------Extract latitudes also
    coors_for_idw   = [(i,j) for i,j in zip(lat_for_idw,lon_for_idw)] #---------Create a list of coordinates (lat lon tuples)

    gmt_0000n       = [gmt_0000[i] for i in neigh5] #---------------------------Find variable values at nearest neighbors
    gmt_0300n       = [gmt_0300[i] for i in neigh5]
    gmt_0600n       = [gmt_0600[i] for i in neigh5]
    gmt_0900n       = [gmt_0900[i] for i in neigh5]
    gmt_1200n       = [gmt_1200[i] for i in neigh5]
    gmt_1500n       = [gmt_1500[i] for i in neigh5]
    gmt_1800n       = [gmt_1800[i] for i in neigh5]
    gmt_2100n       = [gmt_2100[i] for i in neigh5]

    func_gmt_0000   = idw(coors_for_idw, gmt_0000n) #---------------------------Create inverse distance weighing (IDW) interpolation function at nn coordinates
    func_gmt_0300   = idw(coors_for_idw, gmt_0300n)
    func_gmt_0600   = idw(coors_for_idw, gmt_0600n)
    func_gmt_0900   = idw(coors_for_idw, gmt_0900n)
    func_gmt_1200   = idw(coors_for_idw, gmt_1200n)
    func_gmt_1500   = idw(coors_for_idw, gmt_1500n)
    func_gmt_1800   = idw(coors_for_idw, gmt_1800n)
    func_gmt_2100   = idw(coors_for_idw, gmt_2100n)

    interp_gmt_0000 = func_gmt_0000(target) #-----------------------------------Find interpolated hourly par at target
    interp_gmt_0300 = func_gmt_0300(target)
    interp_gmt_0600 = func_gmt_0600(target)
    interp_gmt_0900 = func_gmt_0900(target)
    interp_gmt_1200 = func_gmt_1200(target)
    interp_gmt_1500 = func_gmt_1500(target)
    interp_gmt_1800 = func_gmt_1800(target)
    interp_gmt_2100 = func_gmt_2100(target)